#Relational Databases; SQL syntax

## Objectives

* Review the context of denormalized vs normalized data in relational databases
* Compare and contrast SQL syntax to pandas (and when we should use what)
* Gain insight behind advanced database useage and defined functions in postgres.

## Class Notes

### What are databases?

Databases are a structured data source optimized for efficient retrieval and storage.

* **structured**: we have to pre-define organization strategy
* **retrieval**: the ability to read data out
* **storage**: the ability to write data and save it

### What is a relational database?

Relational databases are traditionally organized in the following manner:

* A database has tables which represent individual entities or objects.
* Tables have predefined schema – rules that tell it what the data will look like.

Each table should have a primary key column:  a unique identifier for that row. Additionally, each table _can_ have a foreign key column: an id that links this to another table.

<img src='' />

In a **normalized** schema, tables are designed to be thin in order to minimize:

1. The amount of repeated information
2. The amount of bytes stored

<img src='prd' />

What if we had designed the database to look this way with one table?

<img src='denormalized_view' />

1. Repeated information is increased; the user information is repeated in each row.
2. There is increased text storage (text bytes are larger than integer bytes)
3. There is no need to join!

The tradeoff between normalized and denormalized data is **speed vs storage**. Storage (for the most part) is the same everywhere.. so let's focus on the speed side. Speed breaks down into _read speed_ and _write speed_.

Of the two data views:

1. Which would we believe to be slower to read but faster to write?
2. Which would we believe to be slower to write but faster to read?


### SQL syntax

SQL (structured query language) is a query language for loading, retrieving, and updating data in relational databases. Most commonly used SQL databases include:

1. Oracle and MySQL
2. SQL Server
3. PostgreSQL

The SQL-like structure is also heavily borrowed in large scale data languages and platforms:

1. Apache Hive
2. Apache Drill (based on Google's Dremel)
3. Spark SQL

So it is important to learn the basics that fit across all platforms!

#### Good syntax

While companies and data teams end up developing their own sense of SQL style, those new to SQL should adopt at least the following style:

1. Keywords are upper case and begin new lines
2. fields in their own lines
3. continuations are indented

This will be explained as we go through examples below. To help make some connections, there will be some python code blocks using pandas syntax to do similar statements to the SQL queries. They'll be labeled ***pandas*** and ***end_pandas*** to clarify where those are.

#### SELECT
Basic usecase for pulling data from the database.

```SQL
SELECT
    col1,
    col2
FROM table
WHERE [some condition];
```

Example
```SQL
SELECT
    poll_title,
    poll_date
FROM polls
WHERE romney_pct > obama_pct;
```

***pandas***
```python
polls[polls.romney_pct > polls.obama_pct][['poll_title', 'poll_date']]
```
***end_pandas***


Notes:

1. The WHERE is optional, though ultimately filtering data is usually the point of querying from a database.
2. You may SELECT as many columns as you'd like, and alias each.

Questions:

1. abc
2. def


#### Aggregations and GROUP BY
In this SELECT style, columns are either group by keys, or aggregations. 

```SQL
SELECT
    col1,
    AVG(col2)
FROM table
GROUP BY col1;
```

Example
```SQL
SELECT
    poll_date,
    AVG(obama_pct)
FROM polls
GROUP BY poll_date;
```

***pandas***
```python
polls.groupby('poll_date').obama_pct.mean()
```
***end_pandas***


Notes:

1. You may groupby and aggregate as many columns as you'd like.
2. Fields that do NOT use aggregations must be in the group by. Some SQL databases will throw errors; others will give you the wrong data.
3. Standard aggregations include `STDDEV, MIN, MAX, COUNT, SUM`; mostly aggregations that can be quickly solved. For example, `MEDIAN` is less often a function, as the solution is more complicated in SQL.

Questions:

1. abc
2. def


#### JOINs 
JOIN is widely used in normalized data in order for us to denormalize the information. Analysts who work in strong relational databases often have half a dozen joins in their queries.

```SQL
SELECT ...
FROM orders
INNER JOIN order_amounts a on a.order_id = orders.id
INNER JOIN order_items i on i.order_id = orders.id
INNER JOIN variants v on v.id = i.variant_id
INNER JOIN products p on p.id = v.product_id
INNER JOIN suppliers s on s.id = v.supplier_id
INNER JOIN addresses ad on ad.addressable_type = 'Supplier' and ad.addressable_id = s.id
...;
```

Basic Example:

```SQL
SELECT
    t1.c1,
    t1.c2,
    t2.c2
FROM t1 
INNER JOIN t2 ON t1.c1 = t2.c2;
```

***pandas***
```python
t1.join(t2, on='c2')
```
***end_pandas***


There are several join types used, despite the above only using one: `INNER JOIN`.

<img src='joins' />

Note that using JOIN introduces potential change in our data context: One to Many and Many to Many relationships.

<img src='relationships' />



#####Troubleshooting JOINs

Make sure that your results are as expected, so consider what the observation is (the row), and rule check other columns:

* is your expected unique column unique?
* Is there duplicate data elsewhere? 

A common check to see if your data is not unique is throwing a HAVING clause in your JOIN.

#### HAVING
Whereas WHERE is used for precomputation, HAVING is a postcomputation clause, filtering the data after the database engine has done the query's work.

```SQL
SELECT
    t1.c1,
    t1.c2,
    AVG(t2.c2)
FROM t1 
INNER JOIN t2 ON t1.c1 = t2.c2
GROUP BY t1.c1, t1.c2
HAVING AVG(t2.c2) > 10;
```

```SQL
SELECT
    poll_date,
    AVG(obama_pct)
FROM polls
GROUP BY poll_date
HAVING AVG(obama_pct) > 50;
```

***pandas***
```python
polls_group = polls.groupby('poll_date').obama_pct.filter(lambda x: x.mean() > 50)
```
***end_pandas***


Note in this context HAVING allows us to filter on the computed column `AVG(t2.c2)` after the GROUP BY has run.

### Extensibility (Postgres coolness!)

All SQL databases are finetuned to audiences with slightly different functionality. Since we are connecting to a postgres database, we can learn and adopt additional functionality not common in others, like MySQL.


#### Partitioning, Window Functions
Window functions allow you to _subgroup_ aggregations. Two common needs for this are:

1. Providing a comparitive summary of averages or other statistical functions against different group bys;
2. `rank()`ing data observations

```SQL
SELECT
    col1,
    col2,
    rank() over (PARTITION BY col ORDER BY col)
FROM table;
```

The following:

```SQL
SELECT
    user_id,
    order_total,
    rank() over (PARTITION BY user_id ORDER BY order_date)
FROM table;
```

Would create a table that looks like this:

```
user_id, order_total, rank()
1      , 100        , 1
2      , 80         , 1
1      , 25         , 2
5      , 70         , 1
1      , 120        , 3
```

Notes:

1. rank() is a specific postgres function for ranking and ordering, though any aggregation will do here.
2. We can use as many window functions as we'd like.
3. Window functions allow us to aggregate in different ways! What would this following SQL query generate?

```SQL
SELECT
    yearid,
    teamid,
    AVG(salary) over (PARTITION BY yearid, teamid),
    AVG(salary) over (PARTITION BY yearid)
FROM salaries
```

#### Subselects (all sql)

There is a lot of additional, great functionality about postgres, like even writing linear regressions:

```
SELECT
    regr_intercept(yearid, LOG(salary)),
    regr_slope(yearid, LOG(salary)),
    regr_r2(yearid, LOG(salary))
FROM salaries
WHERE salary > 0;
```

But often SQL has to be "tricked" into thinking the data is not aggregated, particularly with rank(). We'll subselects to explain this:

```SQL
SELECT col1
FROM (SELECT
    col1,
    col2
    FROM table) table2
```

In this arbitrary example we can at least see that queries can be nested. We don't see much additional functionality here, but imagine in the orders case:


```SQL
SELECT *
FROM (SELECT
        user_id,
        order_total,
        rank() over (PARTITION BY user_id ORDER BY order_date) as "order_number"
    FROM table) orders
WHERE order_number = 2
```

We now get access to the rank in the WHERE (window functions will not work in HAVING due to complexities). You can also join on subselects:

```SQL
SELECT
    users.platform,
    orders.*
FROM users
INNER JOIN (SELECT
        user_id,
        order_total,
        rank() over (PARTITION BY user_id ORDER BY order_date) as "order_number"
    FROM table) orders on users.id = orders.user_id
WHERE order_number = 2
```